In [1]:
import pandas as pd
from selenium import webdriver
from bs4 import BeautifulSoup
import requests
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
import time
from pathos.multiprocessing import ProcessingPool
from pathos.multiprocessing import ProcessPool
import os
import progressbar
import pickle

In [2]:
df = pd.read_csv(r'D:\Google Drive\내 드라이브\LiveCommerse\Total_DataFrame.csv')
df2 = df.sort_values(by='Num_Sub', ascending=False)
df3 = df2[df2['Tour/Experience'] == 1]
df4 = df3[(df3['Food']==0) & (df3['Beauty']==0) & (df3['Life']==0) & (df3['Fashion']==0) & (df3['Kids']==0) & (df3['Tech']==0) & (df3['Hobby/Leisure']==0) & (df3['Culture']==0)]
df5 = df3[(df3['Food']==1) | (df3['Beauty']==1) | (df3['Life']==1) | (df3['Fashion']==1) | (df3['Kids']==1) | (df3['Tech']==1) | (df3['Hobby/Leisure']==1) | (df3['Culture']==1)]
df5.Num_Video = df5['Num_Video'].astype(int)
nontour_df = df5[['Channel_Name', 'Channel_number', 'Num_Video']]

C:\Users\gkfua\AppData\Local\Temp\ipykernel_2888\569083940.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(r'D:\Google Drive\내 드라이브\LiveCommerse\Total_DataFrame.csv')
C:\Users\gkfua\AppData\Local\Temp\ipykernel_2888\569083940.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df5.Num_Video = df5['Num_Video'].astype(int)


In [3]:
with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Complete_list.pkl', 'rb') as f:
    Complete_list = pickle.load(f)
    
Complete_list

[85453, 19279, 91205, 157, 64400, 15871, 74158, 45281, 18394, 74465]

In [4]:
chrome_driver_path = r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'

class MyCrawler:
    def __init__(self, numprocess, path_save):
        self.n_process = numprocess
        self.pool = ProcessPool(nodes=self.n_process)
        self.path_save = path_save
        
    def multi_processing(self, sites):
        def chat_crawler(url):
            # 사용하는 Library Import
            import pandas as pd
            from selenium import webdriver
            from bs4 import BeautifulSoup
            from selenium.webdriver.common.by import By
            from selenium.webdriver.chrome.options import Options
            from selenium.webdriver.chrome.service import Service
            from webdriver_manager.chrome import ChromeDriverManager
            import time
            import os
            
            chrome_driver_path = r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'
            chrome_options = Options()
            chrome_options.add_argument("--headless")
            # driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=chrome_options)
            driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)

            # 원하는 사이트 로드
            driver.get(url)
            save_number = str(url)
            save_number = save_number[44:50]
            
            # 1분 대기 ... 여러 사이트 열어서 시간이 좀 걸림.
            time.sleep(30)

            # 자동으로 2배속 설정.
            buttun = driver.find_element(By.XPATH, '//*[@id="product-root"]/div/div/div/div/div/div[3]/div[2]/div[2]/div/div[3]/div[2]/button[1]')
            buttun.click()
            time.sleep(1)
            buttun.click()
            
            # 반복 횟수 설정.
            repeat_n = 14
            empty = []
            df = pd.DataFrame({'name' : empty, 'chat' : empty})
        
            # 채팅 및 아이디 크롤링.
            for re_n in range(repeat_n):
                soup = BeautifulSoup(driver.page_source, 'html.parser')
                temp_list = soup.select('div.CommentList_animation_area_Ca5J9')
            
                chat_list_temp = []
                for i in temp_list:
                    for j in i:
                        for k in j:
                            chat = k.select('span')
                            chat = str(chat)
                            chat_list_temp.append(chat)

                name_list_temp = []
                for i in temp_list:
                    for j in i:
                        name = j.select('strong')
                        name = str(name)
                        if 'nickname' in name:
                            name_list_temp.append(name)

                chat_list = []
                for i in chat_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        chat_list.append(i)
                    except:
                        continue
                    
                name_list = []
                for i in name_list_temp:
                    try:
                        i = (i.split('<')[1])
                        i = (i.split('>')[1])
                        name_list.append(i)
                    except:
                        continue
                    
                df_temp = pd.DataFrame({ 'name' : name_list, 'chat' : chat_list})
                df = pd.concat([df, df_temp], ignore_index=True)
                df.drop_duplicates(inplace=True)
                df.reset_index(drop=True, inplace=True)
                df.to_csv(self.path_save + '/Video_{}.csv' .format(save_number), index=False)
                time.sleep(200)
        
            # df 형성 및 저장.
            df.drop_duplicates(inplace=True)
            df.reset_index(drop=True, inplace=True)
            df.to_csv(self.path_save + '/Video_{}.csv' .format(save_number), index=False)
            
            driver.quit()
        
        self.pool.map(chat_crawler, sites)

In [5]:
with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Complete_list.pkl', 'rb') as f:
    Complete_list = pickle.load(f)
    
Complete_list

[85453, 19279, 91205, 157, 64400, 15871, 74158, 45281, 18394, 74465]

In [6]:
path = r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df'
Complete_list = []
check_list = []
    
for i in range(len(nontour_df)):
    temp = nontour_df.iloc[i]
    temp_id = temp[0]
    temp_number = temp[1]
    dir_path = path + f'\{temp_id}_{temp_number}'
    
    with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Complete_list.pkl', 'rb') as f:
        Complete_list = pickle.load(f)
    
    if temp_number not in Complete_list:
        chrome_driver_path = r'D:\Google Drive\내 드라이브\chromedriver_win32\chromedriver.exe'
        chrome_options = Options()
        chrome_options.add_argument("--headless")
        driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)
        url = f'https://shoppinglive.naver.com/categories/dc:5?ownerIds={temp_number}'
        driver.get(url)
        time.sleep(3)

        soup = BeautifulSoup(driver.page_source, 'html.parser')
        url_list = soup.select('#content > section.TitleMoreLinkSection_wrap_mp\+Z0.TitleMoreLinkSection_has_last_child_padding_GHBry.TitleMoreLinkSection_has_next_border_z6TVX.TitleMoreLinkSection_tablet_k1mYW > div.VerticalCardList_wrap_responsive_F9HDD.VerticalCardList_tablet_d1tlh.VerticalCardList_is_infinite_V9p37 > div')
        temp_list = []
        
        for j in url_list:
            j = j.select('a')
            j = str(j)
            j = j.split('href="')[1]
            j = j.split('"><div')[0]
            temp_list.append(j)
            
            
        if 19>len(temp_list)>1:
            a = len(temp_list)            
            try:
                os.mkdir(dir_path)
            except:
                print('Aleady Exist')
            
            n = 5
            slice_list = [temp_list[i*n : n*(i+1)] for i in range(0, (len(temp_list) + n -1)//n)]
            k = len(slice_list)
            print(f'Channel {temp_id}_{temp_number}_{a} Crawling start. K = {k}')
            
            
            for p, l in enumerate(slice_list):
                print('k = {} start' .format(p+1))
                print('Processing ...')
                try:
                    Crawler = MyCrawler(len(l), dir_path)
                    Crawler.multi_processing(l)
                    print(l)
                
                    with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Complete_list.pkl', 'wb') as g:
                        pickle.dump(Complete_list, g)
                    print(f'Channel {temp_id}_{temp_number} Crawling End.')

                except:
                    print(f'Fail. Channel = {temp_id}, Number = {temp_number}')
                    print(l)
                    check_list.append(temp_number)
                    with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Check_list.pkl', 'wb') as q:
                        pickle.dump(check_list, q)
                
        else:
            print(f'Check: {temp_id}_{temp_number}_Length Over')
            check_list.append(temp_number)
            with open(r'D:\Google Drive\내 드라이브\LiveCommerse\FUCKUGOOGLE\nonTour_df\Check_list.pkl', 'wb') as q:
                pickle.dump(check_list, q)

C:\Users\gkfua\AppData\Local\Temp\ipykernel_2888\3953064504.py:18: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_driver_path, options=chrome_options)


Aleady Exist
Channel 신세계라이브쇼핑_59307_10 Crawling start. K = 2
k = 1 start
Processing ...
['https://view.shoppinglive.naver.com/replays/585663?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/580259?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/578517?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/544407?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/527794?tr=lim&amp;fm=shoppinglive&amp;sn=home']
Channel 신세계라이브쇼핑_59307 Crawling End.
k = 2 start
Processing ...
['https://view.shoppinglive.naver.com/replays/532394?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/188704?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/162332?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://view.shoppinglive.naver.com/replays/145249?tr=lim&amp;fm=shoppinglive&amp;sn=home', 'https://